In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('/Users/Desktop/dogImages/dogImages/train')
valid_files, valid_targets = load_dataset('/Users/Desktop/dogImages/dogImages/valid')
test_files, test_targets = load_dataset('/Users/Desktop/dogImages/dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %d total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 0 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [13]:
pip install ipywidgets 


The following command must be run outside of the IPython shell:

    $ pip install ipywidgets 

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [17]:
from tqdm import tqdm


In [18]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [19]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:15<00:00, 54.59it/s]


In [2]:
import numpy as np

### TODO: Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('/Users/prasanth/Desktop/BDIA_Assignments/housesalesprediction/train/dog-breed-identification/DogResnet50Data.npz')
train_ResNet50 = bottleneck_features['train']
valid_ResNet50 = bottleneck_features['valid']
test_ResNet50 = bottleneck_features['test']

In [3]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model = Sequential()
ResNet_model.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model.add(Dense(133, activation='softmax'))

ResNet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


# Compile the Model¶


In [42]:
from keras.optimizers import Adam, Adamax

ResNet_model.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

# Train the Model
Here we train the model in the code cell below.



In [7]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs = 30
batch_size = 64

ResNet_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s 76us/step - loss: 0.0092 - acc: 0.9988 - val_loss: 0.5457 - val_acc: 0.8479

Epoch 00001: val_loss improved from inf to 0.54575, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/30
6680/6680 [==============================] - 1s 82us/step - loss: 0.0090 - acc: 0.9987 - val_loss: 0.5489 - val_acc: 0.8479

Epoch 00002: val_loss did not improve from 0.54575
Epoch 3/30
6680/6680 [==============================] - 1s 87us/step - loss: 0.0082 - acc: 0.9990 - val_loss: 0.5498 - val_acc: 0.8515

Epoch 00003: val_loss did not improve from 0.54575
Epoch 4/30
6680/6680 [==============================] - 1s 89us/step - loss: 0.0075 - acc: 0.9990 - val_loss: 0.5529 - val_acc: 0.8503

Epoch 00004: val_loss did not improve from 0.54575
Epoch 5/30
6680/6680 [==============================] - 1s 77us/step - loss: 0.0069 - acc: 0.9985 - val_loss: 0.5421 - val_acc: 

In [8]:
opt = Adamax(lr=0.0002)
epochs = 5
batch_size = 64

ResNet_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/5
6680/6680 [==============================] - 1s 80us/step - loss: 0.0028 - acc: 0.9987 - val_loss: 0.6375 - val_acc: 0.8467

Epoch 00001: val_loss did not improve from 0.54210
Epoch 2/5
6680/6680 [==============================] - 0s 73us/step - loss: 0.0023 - acc: 0.9991 - val_loss: 0.6434 - val_acc: 0.8455

Epoch 00002: val_loss did not improve from 0.54210
Epoch 3/5
6680/6680 [==============================] - 1s 81us/step - loss: 0.0026 - acc: 0.9988 - val_loss: 0.6479 - val_acc: 0.8503

Epoch 00003: val_loss did not improve from 0.54210
Epoch 4/5
6680/6680 [==============================] - 1s 83us/step - loss: 0.0024 - acc: 0.9990 - val_loss: 0.6499 - val_acc: 0.8563

Epoch 00004: val_loss did not improve from 0.54210
Epoch 5/5
6680/6680 [==============================] - 1s 88us/step - loss: 0.0023 - acc: 0.9990 - val_loss: 0.6617 - val_acc: 0.8467

Epoch 00005: val_loss did not improve from 0.54210


# Load the Model with the Best Validation Loss¶


In [21]:
### Load the model weights with the best validation loss.
ResNet_model.load_weights('/Users/Downloads/weights.best_adamax.ResNet50.hdf5')

# Test the Model
Let's try out our model on the test dataset of dog images.

In [23]:
### TODO: Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet_model.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)


Test accuracy: 83.2536%


# Part B - Activation function (10 points)¶


# Tanh

In [29]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model_tanh = Sequential()
ResNet_model_tanh.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model_tanh.add(Dense(133, activation='tanh'))

ResNet_model_tanh.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


Complie the model

In [31]:
from keras.optimizers import Adam, Adamax

ResNet_model_tanh.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [33]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs = 30
batch_size = 64

ResNet_model_tanh.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s 165us/step - loss: 5.4356 - acc: 0.0094 - val_loss: 5.3884 - val_acc: 0.0084

Epoch 00001: val_loss improved from inf to 5.38837, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/30
6680/6680 [==============================] - 1s 79us/step - loss: 5.2983 - acc: 0.0099 - val_loss: 5.4253 - val_acc: 0.0084

Epoch 00002: val_loss did not improve from 5.38837
Epoch 3/30
6680/6680 [==============================] - 1s 102us/step - loss: 5.2911 - acc: 0.0096 - val_loss: 5.6198 - val_acc: 0.0108

Epoch 00003: val_loss did not improve from 5.38837
Epoch 4/30
6680/6680 [==============================] - 1s 97us/step - loss: 6.2671 - acc: 0.0096 - val_loss: 11.1226 - val_acc: 0.0120

Epoch 00004: val_loss did not improve from 5.38837
Epoch 5/30
6680/6680 [==============================] - 0s 71us/step - loss: 10.0786 - acc: 0.0129 - val_loss: 9.4709 - val_a

In [36]:
ResNet_model_tanh.load_weights('/Users/Downloads/weights.best_adamax.ResNet50.hdf5')

In [37]:
### TODO: Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet_model_tanh.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)


Test accuracy: 1.7943%


# relu

In [30]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model_relu = Sequential()
ResNet_model_relu.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model_relu.add(Dense(133, activation='relu'))

ResNet_model_relu.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_7 ( (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [34]:
ResNet_model_relu.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [35]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs = 30
batch_size = 64

ResNet_model_tanh.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s 80us/step - loss: 9.1202 - acc: 0.0180 - val_loss: 9.0702 - val_acc: 0.0156

Epoch 00001: val_loss improved from inf to 9.07018, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/30
6680/6680 [==============================] - 1s 78us/step - loss: 9.2496 - acc: 0.0186 - val_loss: 9.3575 - val_acc: 0.0180

Epoch 00002: val_loss did not improve from 9.07018
Epoch 3/30
6680/6680 [==============================] - 1s 94us/step - loss: 9.3692 - acc: 0.0175 - val_loss: 9.1054 - val_acc: 0.0132

Epoch 00003: val_loss did not improve from 9.07018
Epoch 4/30
6680/6680 [==============================] - 1s 94us/step - loss: 9.4826 - acc: 0.0195 - val_loss: 9.5295 - val_acc: 0.0144

Epoch 00004: val_loss did not improve from 9.07018
Epoch 5/30
6680/6680 [==============================] - 1s 82us/step - loss: 9.5462 - acc: 0.0172 - val_loss: 9.5116 - val_acc: 

In [38]:
ResNet_model_relu.load_weights('/Users/Downloads/weights.best_adamax.ResNet50.hdf5')

In [39]:
### TODO: Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
ResNet50_predictions_relu = [np.argmax(ResNet_model_relu.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy_relu = 100*np.sum(np.array(ResNet50_predictions_relu)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions_relu)
print('Test accuracy: %.4f%%' % test_accuracy_relu)


Test accuracy: 1.9139%


### Accuracy for tanh is 1.7943%
### Accuracy for relu is 1.9139
### Accuracy for Softmax is  83.2536%




# Part C - Cost function (10 points)¶


In [47]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model_hinge = Sequential()
ResNet_model_hinge.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model_hinge.add(Dense(133, activation='relu'))

ResNet_model_hinge.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_9 ( (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [50]:
ResNet_model_hinge.compile(loss='hinge', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [52]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs = 30
batch_size = 64

ResNet_model_hinge.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s 139us/step - loss: 0.9935 - acc: 0.0114 - val_loss: 0.9929 - val_acc: 0.0060

Epoch 00001: val_loss improved from inf to 0.99292, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/30
6680/6680 [==============================] - 1s 78us/step - loss: 0.9929 - acc: 0.0094 - val_loss: 0.9929 - val_acc: 0.0060

Epoch 00002: val_loss improved from 0.99292 to 0.99286, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 3/30
6680/6680 [==============================] - 1s 92us/step - loss: 0.9929 - acc: 0.0094 - val_loss: 0.9929 - val_acc: 0.0060

Epoch 00003: val_loss did not improve from 0.99286
Epoch 4/30
6680/6680 [==============================] - 1s 81us/step - loss: 0.9929 - acc: 0.0094 - val_loss: 0.9929 - val_acc: 0.0060

Epoch 00004: val_loss did not improve from 0.99286
Epoch 5/30
6680/6680 [========================

# Load the Model with the Best Validation Loss¶


In [55]:
### Load the model weights with the best validation loss.
ResNet_model_hinge.load_weights('/Users/Downloads/weights.best_adamax.ResNet50.hdf5')

In [59]:
### TODO: Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
ResNet50_predictions_hinge = [np.argmax(ResNet_model_hinge.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy_hinge = 100*np.sum(np.array(ResNet50_predictions_hinge)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions_hinge)

In [60]:
print('Test accuracy: %.4f%%' % test_accuracy_hinge)


Test accuracy: 0.5981%


### We get accuracy of 50 percent

# Part D - Epochs (10 points)¶
# Part F - Network Architecture (10 points)¶



In [64]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model_epoch = Sequential()
ResNet_model_epoch.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model_epoch.add(Dense(250, activation='softmax'))

ResNet_model_epoch.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_11  (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 250)               512250    
Total params: 512,250
Trainable params: 512,250
Non-trainable params: 0
_________________________________________________________________


In [65]:
from keras.optimizers import Adam, Adamax

ResNet_model_epoch.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [66]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs_2 = 100
batch_size = 64

ResNet_model_tanh.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs_2, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/100
6680/6680 [==============================] - 2s 243us/step - loss: 8.1194 - acc: 0.0099 - val_loss: 8.1274 - val_acc: 0.0132

Epoch 00001: val_loss improved from inf to 8.12736, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/100
6680/6680 [==============================] - 1s 76us/step - loss: 8.1194 - acc: 0.0100 - val_loss: 8.1276 - val_acc: 0.0156

Epoch 00002: val_loss did not improve from 8.12736
Epoch 3/100
6680/6680 [==============================] - 1s 90us/step - loss: 8.0929 - acc: 0.0106 - val_loss: 8.1655 - val_acc: 0.0156

Epoch 00003: val_loss did not improve from 8.12736
Epoch 4/100
6680/6680 [==============================] - 1s 95us/step - loss: 8.1052 - acc: 0.0106 - val_loss: 8.2820 - val_acc: 0.0132

Epoch 00004: val_loss did not improve from 8.12736
Epoch 5/100
6680/6680 [==============================] - 1s 98us/step - loss: 8.1877 - acc: 0.0105 - val_loss: 8.2824 - val

6680/6680 [==============================] - 1s 87us/step - loss: 8.0689 - acc: 0.0103 - val_loss: 7.9145 - val_acc: 0.0156

Epoch 00084: val_loss did not improve from 7.60621
Epoch 85/100
6680/6680 [==============================] - 1s 75us/step - loss: 8.1537 - acc: 0.0100 - val_loss: 8.2813 - val_acc: 0.0084

Epoch 00085: val_loss did not improve from 7.60621
Epoch 86/100
6680/6680 [==============================] - 1s 104us/step - loss: 8.2524 - acc: 0.0087 - val_loss: 8.2232 - val_acc: 0.0120

Epoch 00086: val_loss did not improve from 7.60621
Epoch 87/100
6680/6680 [==============================] - 1s 121us/step - loss: 8.2161 - acc: 0.0099 - val_loss: 8.3033 - val_acc: 0.0072

Epoch 00087: val_loss did not improve from 7.60621
Epoch 88/100
6680/6680 [==============================] - 1s 111us/step - loss: 8.2595 - acc: 0.0090 - val_loss: 8.2039 - val_acc: 0.0084

Epoch 00088: val_loss did not improve from 7.60621
Epoch 89/100
6680/6680 [==============================] - 1s 101u

# Epochs updated to 100 its accuracy is 80%

##  Changing th dense layer to 250 fgrom 133 23 get an accuracy of 76 percent

# Change of optimiser to SGD from Adam

In [102]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model_optimiser = Sequential()
ResNet_model_optimiser.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))

ResNet_model_optimiser.add(Dense(133, activation='softmax'))

ResNet_model_optimiser.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_17  (None, 2048)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [103]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


# Change of optimiser to SGD from Adam

In [104]:
from keras.optimizers import Adam, Adamax

ResNet_model_optimiser.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [105]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='/Users/Downloads/weights.best_adamax.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

epochs_2 = 100
batch_size = 64

ResNet_model_optimiser.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs_2, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/100
6680/6680 [==============================] - 1s 103us/step - loss: 4.2632 - acc: 0.1398 - val_loss: 3.3179 - val_acc: 0.3461

Epoch 00001: val_loss improved from inf to 3.31791, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 2/100
6680/6680 [==============================] - 0s 67us/step - loss: 2.6725 - acc: 0.4692 - val_loss: 2.2832 - val_acc: 0.5461

Epoch 00002: val_loss improved from 3.31791 to 2.28316, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 3/100
6680/6680 [==============================] - 1s 80us/step - loss: 1.8671 - acc: 0.6419 - val_loss: 1.7485 - val_acc: 0.6491

Epoch 00003: val_loss improved from 2.28316 to 1.74855, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 4/100
6680/6680 [==============================] - 0s 70us/step - loss: 1.4315 - acc: 0.7274 - val_loss: 1.4514 - val_acc: 0.6862

Epoch 00004:

6680/6680 [==============================] - 0s 73us/step - loss: 0.1338 - acc: 0.9939 - val_loss: 0.5481 - val_acc: 0.8311

Epoch 00062: val_loss improved from 0.54842 to 0.54810, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 63/100
6680/6680 [==============================] - 0s 65us/step - loss: 0.1316 - acc: 0.9939 - val_loss: 0.5471 - val_acc: 0.8323

Epoch 00063: val_loss improved from 0.54810 to 0.54709, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 64/100
6680/6680 [==============================] - 0s 65us/step - loss: 0.1294 - acc: 0.9943 - val_loss: 0.5445 - val_acc: 0.8323

Epoch 00064: val_loss improved from 0.54709 to 0.54448, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 65/100
6680/6680 [==============================] - 0s 65us/step - loss: 0.1276 - acc: 0.9943 - val_loss: 0.5438 - val_acc: 0.8311

Epoch 00065: val_loss improved from 0.54448 to 0.54384, saving mo


Epoch 00094: val_loss improved from 0.52733 to 0.52688, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 95/100
6680/6680 [==============================] - 0s 68us/step - loss: 0.0867 - acc: 0.9972 - val_loss: 0.5272 - val_acc: 0.8335

Epoch 00095: val_loss did not improve from 0.52688
Epoch 96/100
6680/6680 [==============================] - 0s 64us/step - loss: 0.0858 - acc: 0.9979 - val_loss: 0.5262 - val_acc: 0.8359

Epoch 00096: val_loss improved from 0.52688 to 0.52623, saving model to /Users/prasanth/Downloads/weights.best_adamax.ResNet50.hdf5
Epoch 97/100
6680/6680 [==============================] - 0s 67us/step - loss: 0.0849 - acc: 0.9979 - val_loss: 0.5263 - val_acc: 0.8371

Epoch 00097: val_loss did not improve from 0.52623
Epoch 98/100
6680/6680 [==============================] - 0s 67us/step - loss: 0.0841 - acc: 0.9979 - val_loss: 0.5264 - val_acc: 0.8347

Epoch 00098: val_loss did not improve from 0.52623
Epoch 99/100
6680/6680 [======

#  MIT License: 
Copyright 2019 Spoorthi Rao

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.